In [1]:
#csv파일이랑 audio폴더 test 양만큼 개줄여서 디버그하며 할 것

import os
import librosa
import librosa.display
import IPython.display
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
import pandas as pd
import csv

import scipy
from scipy.stats import skew
from tqdm import tqdm, tqdm_pandas

tqdm.pandas()

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.svm import SVC

In [2]:

audio_train_files = os.listdir('./audio_train/')
audio_test_files = os.listdir('./audio_test/')
train=pd.read_csv('./file_name.csv')

In [3]:
#print(audio_train_files)
#print(audio_test_files)

In [4]:
def get_mfcc(name, path):
    data, _ = librosa.core.load(path+name) #엑셀에 .wav까지 붙여 저장해야됨 안그러면 path+name+'.wav' 이렇게해야
    print(data,_)
    #data, _ = librosa.load(path)
    try:
        #ft1 = librosa.feature.mfcc(data, sr = SAMPLE_RATE, n_mfcc=30)
        ft1 = librosa.feature.mfcc(data, n_mfcc=30) #이게핵심 Sample_rate를 어떻게 조절하느냐
        ft2 = librosa.feature.zero_crossing_rate(data)[0] # n_mfcc 값을 어떻게 조절하느냐에 따라 Feature개수가 달리 뽑힘
        ft3 = librosa.feature.spectral_rolloff(data)[0]
        ft4 = librosa.feature.spectral_centroid(data)[0]
        ft5 = librosa.feature.spectral_contrast(data)[0]
        ft6 = librosa.feature.spectral_bandwidth(data)[0]
        ft1_trunc = np.hstack((np.mean(ft1, axis=1), np.std(ft1, axis=1), skew(ft1, axis = 1), np.max(ft1, axis = 1), np.median(ft1, axis = 1), np.min(ft1, axis = 1)))
        ft2_trunc = np.hstack((np.mean(ft2), np.std(ft2), skew(ft2), np.max(ft2), np.median(ft2), np.min(ft2)))
        ft3_trunc = np.hstack((np.mean(ft3), np.std(ft3), skew(ft3), np.max(ft3), np.median(ft3), np.min(ft3)))
        ft4_trunc = np.hstack((np.mean(ft4), np.std(ft4), skew(ft4), np.max(ft4), np.median(ft4), np.min(ft4)))
        ft5_trunc = np.hstack((np.mean(ft5), np.std(ft5), skew(ft5), np.max(ft5), np.median(ft5), np.min(ft5)))
        ft6_trunc = np.hstack((np.mean(ft6), np.std(ft6), skew(ft6), np.max(ft6), np.median(ft6), np.max(ft6)))
        return pd.Series(np.hstack((ft1_trunc, ft2_trunc, ft3_trunc, ft4_trunc, ft5_trunc, ft6_trunc)))
    except:
        print('bad file')
        return pd.Series([0]*210)

In [5]:
train_data = pd.DataFrame()
train_data['fname'] = train['fname']
test_data=pd.DataFrame()
test_data['fname']=audio_test_files

In [6]:
#print(train_data)
#print(test_data)


In [7]:
train_data=train_data['fname'].progress_apply(get_mfcc,path='./audio_train/')
print('done loading train mfcc')
test_data = test_data['fname'].progress_apply(get_mfcc, path='./audio_test/')
print('done loading test mfcc')



  0%|                                                                                          | 0/599 [00:00<?, ?it/s]

[0.06973912 0.10266001 0.09116188 ... 0.08305359 0.08018959 0.08612607] 22050


  0%|▎                                                                                 | 2/599 [00:01<06:13,  1.60it/s]

[-0.02710845 -0.03993477 -0.03530764 ... -0.04230448 -0.0414599
 -0.04527015] 22050


  1%|▍                                                                                 | 3/599 [00:02<06:57,  1.43it/s]

...

 99%|█████████████████████████████████████████████████████████████████████████████████ | 84/85 [01:08<00:00,  1.21it/s]

[ 0.          0.          0.         ... -0.22099693 -0.21257466
 -0.23776534] 22050


100%|██████████████████████████████████████████████████████████████████████████████████| 85/85 [01:09<00:00,  1.21it/s]

[ 0.          0.          0.         ... -0.05103435 -0.05238643
 -0.05545169] 22050


done loading test mfcc


In [8]:
train_data['fname'] = train['fname']
train_data['label'] = train['label']

test_data['fname'] = audio_test_files
test_data['label'] = np.zeros((len(audio_test_files)))

In [9]:
#print(train_data)
#print(test_data)

In [10]:
X = train_data.drop(['label', 'fname'], axis=1)
feature_names = list(X.columns)
X = X.values
labels = np.sort(np.unique(train_data.label.values))
num_class = len(labels)
c2i = {}
i2c = {}
for i, c in enumerate(labels):
    c2i[c] = i
    i2c[i] = c
y = np.array([c2i[x] for x in train_data.label.values])

#print(X)
#y는 train 데이터들이 라벨링한 것
print(y)

[0 0 1 2 3 3 3 3 3 3 0 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 4 3 3 3 3 3 3 3 3 3 3 3 5 5 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 5 5
 3 3 3 3 3 3 3 3 0 3 3 3 3 3 1 1 6 6 6 6 6 6 6 6 6 1 3 3 3 3 3 3 1 1 1 3 0
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 0 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 6 6 6 6 6 6 6 6 6 6 6 6 6 1 0 0 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 6 6 6 6 3 3
 3 3 3 3 3 3 3 1 1 1 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 0 7 7 7 0 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 1 1 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 1 1 1 1 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 7 7 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 0 0 0 0 3 3 3
 3 3 7 7 5 5 5 3 3 3 3 3 

In [11]:
X_test = test_data.drop(['label', 'fname'], axis=1)
X_test = X_test.values

In [12]:
print(X.shape)
print(X_test.shape)
#test의 라벨링은 어디서?

(599, 210)
(85, 210)


In [13]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)
#이놈이 뭘까

In [14]:
#pca = PCA(n_components=65).fit(X_scaled)
#65땜에 오류뜸
pca = PCA(n_components=65).fit(X_scaled)
#이 값 고쳐야
X_pca = pca.transform(X_scaled)
X_test_pca = pca.transform(X_test_scaled)

print(sum(pca.explained_variance_ratio_)) 

0.9485330413910205


In [15]:
# Fit an SVM model
X_train, X_val, y_train, y_val = train_test_split(X_pca, y, test_size = 0.2, random_state = 42, shuffle = True)
#test_size와 random_state 확인

#train 관련
#print(X_train)
#print(y_train) - train 라벨링

#train 내에서 쪼개는 것으로 추측됨 test 0.2가 100개중 20개로 맞네
#test를 따로 만드는게 아니라 train 내에서 일부를 test에 사용하여 정확도 측정
print(X_val)
print(y_val)

clf = SVC(kernel = 'rbf', probability=True)

clf.fit(X_train, y_train)

print(accuracy_score(clf.predict(X_val), y_val))

[[-7.84837181e+00  4.08761833e+00  1.92600495e+00 ...  7.79534620e-01
  -4.43196492e-01  3.48754183e-01]
 [-1.00013680e+01  2.56574213e+00  1.36577499e+00 ...  2.92318966e-01
   1.45759369e-01  1.86326452e-01]
 [ 9.12132968e-01  2.24720265e+00 -3.78759332e+00 ...  9.79688028e-01
   6.82013525e-01  3.30805343e-01]
 ...
 [-3.74352064e+00 -2.73003407e+00  1.07282330e+00 ...  5.13664097e-01
  -9.10511019e-02 -1.62329537e-01]
 [ 2.67848851e+00 -7.30593037e+00  3.94198005e+00 ... -4.56912215e-01
   3.52476589e-03 -9.88331500e-01]
 [-6.25764192e-01 -9.97515843e+00  5.32709852e+00 ... -1.62210549e-01
   5.31608769e-01  6.82025753e-01]]
[0 3 3 3 3 3 0 3 3 3 3 3 3 3 3 3 3 6 3 3 1 3 3 3 3 3 5 3 3 3 3 3 0 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 6 3 3 3 3 3 3 3 3 3 3 3 3 3 7 3 3 3
 3 3 3 3 3 3 3 3 3 3 5 3 3 0 3 6 3 3 3 3 3 3 3 3 3 3 6 6 3 3 3 3 3 3 1 3 3
 3 3 3 3 3 3 3 3 3]


c:\users\dong\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.9083333333333333


In [16]:
C_grid = [0.001, 0.01, 0.1, 1, 10]
gamma_grid = [0.001, 0.01, 0.1, 1, 10]
param_grid = {'C': C_grid, 'gamma' : gamma_grid}

#grid = GridSearchCV(SVC(kernel='rbf'), param_grid, cv = 3, scoring = "accuracy")
#이부분 수정해야
#데이터 갯수 확 느니까 바로먹음
grid = GridSearchCV(SVC(kernel='rbf'), param_grid, cv = 3, scoring = "accuracy")
grid.fit(X_train, y_train)

# Find the best model
print(grid.best_score_)

print(grid.best_params_)

print(grid.best_estimator_)

c:\users\dong\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)
c:\users\dong\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


0.9311064718162839
{'C': 10, 'gamma': 0.01}
SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [17]:
clf = SVC(kernel = 'rbf', C = 10, gamma = 0.01, probability=True)

clf.fit(X_train, y_train)

print(accuracy_score(clf.predict(X_val), y_val))

0.9333333333333333


In [18]:
# Fit the entire training sets
clf.fit(X_pca, y)


SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)